[View in Colaboratory](https://colab.research.google.com/github/fabiansd/AI-workshop/blob/master/PricePrediction.ipynb)

In [10]:
!pip install kaggle-cli
!kg download -u fabiansd  -p thisisnotmyrealpassword -c competitive-data-science-predict-future-sales
!gzip -d sales_train.csv.gz
!gzip -d sample_submission.csv.gz
!gzip -d test.csv.gz
!gzip -d items.csv.gz
!gzip -d item_categories.csv.gz
!gzip -d shops.csv.gz
!ls

/usr/local/lib/python2.7/dist-packages/mechanicalsoup/browser.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /usr/local/lib/python2.7/dist-packages/mechanicalsoup/browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  response.content, **soup_config)
downloading https://www.kaggle.com/c/competitive-data-science-predict-future-sales/download/item_categories.csv

item_categories.csv already downloaded !
downloading https://www.kaggle.com/c/competitive-data-science-predict-future-sales/download/items.csv

items.csv already downloaded !
downloading https://www.kaggle.com/c/competitive-data-science-predict-future-sales/do

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sales_train = pd.read_csv('sales_train.csv')
sample_submission = pd.read_csv('sample_submission.csv')
test = pd.read_csv('test.csv')
items = pd.read_csv('items.csv')
items_cat = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')

In [36]:
#Generate data time
sales_train['date'] = pd.to_datetime(sales_train['date'], format='%d.%m.%Y')
sales_train['month'] = sales_train['date'].dt.month
sales_train['year'] = sales_train['date'].dt.year

#Merge dataframes
sales_train = pd.merge(sales_train, items, on='item_id')
sales_train = pd.merge(sales_train, items_cat, on='item_category_id')
sales_train = pd.merge(sales_train, shops, on='shop_id')

#rearrange order
rearrange = ['date','month','year','date_block_num','item_id','item_name','item_price','item_cnt_day','item_category_id','item_category_name','shop_id','shop_name']
train = sales_train[rearrange]
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,year
0,2013-01-02,0,59,22154,999.00,1.0,1,2013
1,2013-01-03,0,25,2552,899.00,1.0,1,2013
2,2013-01-05,0,25,2552,899.00,-1.0,1,2013
3,2013-01-06,0,25,2554,1709.05,1.0,1,2013
4,2013-01-15,0,25,2555,1099.00,1.0,1,2013


In [42]:
train.describe()

,month,year,date_block_num,item_id,item_price,item_cnt_day,item_category_id,shop_id
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,6.247717e+00,2.013777e+03,1.456991e+01,1.019723e+04,8.908532e+02,1.242641e+00,4.000138e+01,3.300173e+01
std,3.536219e+00,7.684790e-01,9.422988e+00,6.324297e+03,1.729800e+03,2.618834e+00,1.710076e+01,1.622697e+01
min,1.000000e+00,2.013000e+03,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01,0.000000e+00,0.000000e+00
25%,3.000000e+00,2.013000e+03,7.000000e+00,4.476000e+03,2.490000e+02,1.000000e+00,2.800000e+01,2.200000e+01
50%,6.000000e+00,2.014000e+03,1.400000e+01,9.343000e+03,3.990000e+02,1.000000e+00,4.000000e+01,3.100000e+01
75%,9.000000e+00,2.014000e+03,2.300000e+01,1.568400e+04,9.990000e+02,1.000000e+00,5.500000e+01,4.700000e+01
max,1.200000e+01,2.015000e+03,3.300000e+01,2.216900e+04,3.079800e+05,2.169000e+03,8.300000e+01,5.900000e+01
